In [56]:
import numpy as np
import pandas as pd
train = pd.read_csv("labeledTrainData.tsv", header = 0, delimiter = '\t')
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [57]:
import re

In [58]:
from collections import Counter

In [59]:
vocab = Counter()
for twit in train['review']:
    for word in twit.split():
        vocab[word] += 1
vocab.most_common(20)

[('the', 287032),
 ('a', 155096),
 ('and', 152664),
 ('of', 142972),
 ('to', 132568),
 ('is', 103228),
 ('in', 85580),
 ('I', 65973),
 ('that', 64560),
 ('this', 57196),
 ('it', 54429),
 ('/><br', 50935),
 ('was', 46698),
 ('as', 42510),
 ('with', 41721),
 ('for', 41070),
 ('but', 33790),
 ('The', 33762),
 ('on', 30766),
 ('movie', 30500)]

In [60]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Asus
[nltk_data]     K43E\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [61]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

print(stop)
vocab_reduced = Counter()
for w, c in vocab.items():
    if not w.lower() in stop:
        vocab_reduced[w] = c

vocab_reduced.most_common(20)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[('/><br', 50935),
 ('movie', 30500),
 ('film', 27397),
 ('one', 20688),
 ('like', 18133),
 ('would', 11922),
 ('good', 11435),
 ('really', 10815),
 ('even', 10607),
 ('see', 10155),
 ('-', 9355),
 ('get', 8777),
 ('story', 8526),
 ('much', 8507),
 ('time', 7764),
 ('make', 7485),
 ('could', 7462),
 ('also', 7422),
 ('first', 7339),
 ('people', 7335)]

In [62]:
def preprocessor(text):
    """ Return a cleaned version of text
    """

    text = re.sub('<[^>]*>', '', text)

    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)

    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text


print(preprocessor('I like it --- :.; :))))'))

i like it  :)


In [63]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()


def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer('Hi there, I am loving this, like with a lot of love'))
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

['Hi', 'there,', 'I', 'am', 'loving', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']
['Hi', 'there,', 'I', 'am', 'love', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']


In [64]:
from sklearn.model_selection import train_test_split

X = train['sentiment']
y = train['review']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [65]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

TypeError: expected string or bytes-like object

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))